In [5]:
# ============================================================
# CELL 1A: Importing ans Installing modules
# ============================================================

import tensorflow as tf
import numpy as np

print("TensorFlow:", tf.__version__)
print("NumPy:", np.__version__)

!pip install -q "mediapipe==0.10.14" "opencv-python-headless==4.8.1.78" pandas matplotlib tqdm

TensorFlow: 2.19.0
NumPy: 2.0.2


In [7]:
# ============================================================
# CELL 1B: Initialize TPU/GPU Strategy
# ============================================================

import tensorflow as tf

# Try TPU, fallback to GPU/CPU
USE_TPU = False

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    USE_TPU = True
    print(f"✅ Running on TPU: {tpu.cluster_spec().as_dict()['worker']}")
except (ValueError, tf.errors.NotFoundError):
    # No TPU found, use GPU if available
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        strategy = tf.distribute.MirroredStrategy()
        print(f"✅ Running on {len(gpus)} GPU(s)")
    else:
        strategy = tf.distribute.get_strategy()  # Default (single CPU/GPU)
        print("✅ Running on CPU/single GPU")

print(f"Strategy: {strategy.__class__.__name__}")
print(f"Number of devices: {strategy.num_replicas_in_sync}")

✅ Running on 1 GPU(s)
Strategy: MirroredStrategy
Number of devices: 1


In [8]:
# ============================================================
# CELL 2: Mount Drive & Set Project Directory
# ============================================================

from google.colab import drive
drive.mount('/content/gdrive')

import os

PROJECT_ROOT = "/content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/python_training"
os.makedirs(PROJECT_ROOT, exist_ok=True)
os.chdir(PROJECT_ROOT)

print("Current directory:", os.getcwd())

# Do NOT re-install mediapipe here; it is already installed in Cell 1.
# If you really need extra libs later, install them separately.
import mediapipe as mp
print("MediaPipe version:", mp.__version__)

Mounted at /content/gdrive
Current directory: /content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/python_training
MediaPipe version: 0.10.14


In [9]:
# ============================================================
# CELL 3: Define Target Gestures
# ============================================================

# Hand gestures from HaGRID (model labels)
# Mapping comments show how you will use them in the app.
HAND_GESTURES = {
    "call":    "call",    # VOICE_ASSISTANT
    "like":    "like",    # INCREASE_VOLUME
    "dislike": "dislike", # DECREASE_VOLUME
    "fist":    "fist",    # PLAY_PAUSE
    "palm":    "palm",    # neutral / no-op
    "peace":   "peace",   # OPEN_CAMERA
    "rock":    "rock",    # SCREENSHOT
}

print("Hand gestures to train:", list(HAND_GESTURES.keys()))

ALLOWED_HAND_GESTURES = set(HAND_GESTURES.keys())

Hand gestures to train: ['call', 'like', 'dislike', 'fist', 'palm', 'peace', 'rock']


In [11]:
# ============================================================
# CELL 4: Dataset Configuration Flags
# ============================================================

# Control which datasets to use for training
USE_PUBLIC_DATA = True   # Include HaGRID public dataset
USE_MANUAL_DATA = True   # Include manually collected data

print("Dataset Configuration:")
print(f"  • Use HaGRID Public Data: {USE_PUBLIC_DATA}")
print(f"  • Use Manual Collected Data: {USE_MANUAL_DATA}")

if not USE_PUBLIC_DATA and not USE_MANUAL_DATA:
    print("\n⚠️ WARNING: Both datasets disabled! Must enable at least one.")

Dataset Configuration:
  • Use HaGRID Public Data: True
  • Use Manual Collected Data: True


In [12]:
# ============================================================
# CELL 5: Extract HaGRID subset (Optimized + Stratified)
# ============================================================

import os
import zipfile
from tqdm import tqdm
import time
import random

# Configuration
HAGRID_ZIP_PATH = "/content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/datasets/hagrid-classification-512p.zip"
HAGRID_SUBSET_DIR = os.path.join(PROJECT_ROOT, "hagrid_subset")

# NEW: Optimal sample size with stratified sampling
SAMPLES_PER_GESTURE = 1500  # ← Increased from 500
USE_STRATIFIED_SAMPLING = True  # ← Ensures diversity

# Skip if not using public data
if not USE_PUBLIC_DATA:
    print("=" * 60)
    print("⏭️  SKIPPING HAGRID EXTRACTION")
    print("=" * 60)
    print("USE_PUBLIC_DATA = False, skipping HaGRID dataset extraction")
    print("\n✓ Moving to next step...")
else:
    print("=" * 60)
    print("EXTRACTING HAGRID SUBSET (OPTIMIZED)")
    print("=" * 60)
    print(f"Samples per gesture: {SAMPLES_PER_GESTURE}")
    print(f"Stratified sampling: {USE_STRATIFIED_SAMPLING}")

    # Check if already extracted
    already_extracted = True
    if os.path.exists(HAGRID_SUBSET_DIR):
        for gesture_name in HAND_GESTURES.keys():
            gesture_dir = os.path.join(HAGRID_SUBSET_DIR, gesture_name)
            if not os.path.exists(gesture_dir):
                already_extracted = False
                break
            existing_count = len([f for f in os.listdir(gesture_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
            if existing_count < SAMPLES_PER_GESTURE:
                already_extracted = False
                break
    else:
        already_extracted = False

    if already_extracted:
        print(f"\n✓ HaGRID subset already extracted to {HAGRID_SUBSET_DIR}")
        print("Skipping extraction...")
    else:
        # Check if ZIP exists
        if not os.path.exists(HAGRID_ZIP_PATH):
            print("⚠️ HaGRID zip not found at:", HAGRID_ZIP_PATH)
            print("\nOptions:")
            print("1. Set USE_PUBLIC_DATA = False to skip HaGRID")
            print("2. Download HaGRID and place at:", HAGRID_ZIP_PATH)
        else:
            os.makedirs(HAGRID_SUBSET_DIR, exist_ok=True)

            # Function to extract with retry
            def extract_with_retry(zf, member, dest_path, max_retries=3):
                """Extract a single file with retry logic for Drive disconnections."""
                for attempt in range(max_retries):
                    try:
                        with zf.open(member) as src, open(dest_path, "wb") as dst:
                            dst.write(src.read())
                        return True
                    except OSError as e:
                        if "Transport endpoint" in str(e) or "errno 107" in str(e).lower():
                            if attempt < max_retries - 1:
                                print(f"\n⚠️ Connection lost, retrying ({attempt + 1}/{max_retries})...")
                                time.sleep(2)
                                continue
                            else:
                                print(f"\n❌ Failed after {max_retries} attempts: {member.filename}")
                                return False
                        else:
                            raise
                return False

            print(f"\nExtracting {SAMPLES_PER_GESTURE} samples per gesture from ZIP...")

            try:
                with zipfile.ZipFile(HAGRID_ZIP_PATH, "r") as zf:
                    all_names = zf.namelist()
                    print("Total entries in zip:", len(all_names))

                    for gesture_name in HAND_GESTURES.keys():
                        gesture_dir = os.path.join(HAGRID_SUBSET_DIR, gesture_name)
                        os.makedirs(gesture_dir, exist_ok=True)

                        # Find images for this gesture
                        pattern = f"hagrid-classification-512p/{gesture_name}/"
                        candidates = [
                            name for name in all_names
                            if pattern in name and name.lower().endswith(('.jpg', '.png', '.jpeg'))
                        ]

                        if not candidates:
                            print(f"\n⚠️ No images found for gesture: {gesture_name}")
                            continue

                        print(f"\n{gesture_name}: found {len(candidates)} images under '{pattern}'")

                        # NEW: Stratified sampling for better diversity
                        if USE_STRATIFIED_SAMPLING and len(candidates) > SAMPLES_PER_GESTURE:
                            # Shuffle with fixed seed for reproducibility
                            random.Random(42).shuffle(candidates)

                            # Take evenly spaced samples across the dataset
                            step = len(candidates) / SAMPLES_PER_GESTURE
                            selected = [candidates[int(i * step)] for i in range(SAMPLES_PER_GESTURE)]

                            print(f"  Using stratified sampling (step={step:.1f})")
                        else:
                            # Fallback to first N samples
                            selected = candidates[:SAMPLES_PER_GESTURE]

                        successful = 0
                        failed = 0

                        for img_name in tqdm(selected, desc=f"Extracting {gesture_name}"):
                            base_name = os.path.basename(img_name)
                            dest_path = os.path.join(gesture_dir, base_name)

                            # Skip if already extracted
                            if os.path.exists(dest_path):
                                successful += 1
                                continue

                            # Extract with retry
                            if extract_with_retry(zf, img_name, dest_path):
                                successful += 1
                            else:
                                failed += 1

                        print(f"  ✓ {successful} successful, {failed} failed")

                print("\n✓ HaGRID subset extraction complete!")

            except OSError as e:
                if "Transport endpoint" in str(e) or "errno 107" in str(e).lower():
                    print("\n" + "=" * 60)
                    print("❌ GOOGLE DRIVE CONNECTION ERROR")
                    print("=" * 60)
                    print("\nThe connection to Google Drive was lost during extraction.")
                    print("\n🔧 SOLUTIONS:")
                    print("\n1. COPY ZIP TO COLAB DISK (RECOMMENDED):")
                    print("   Run this in a new cell:")
                    print("   ```")
                    print("   import shutil")
                    print(f"   shutil.copy('{HAGRID_ZIP_PATH}', '/tmp/hagrid.zip')")
                    print("   HAGRID_ZIP_PATH = '/tmp/hagrid.zip'")
                    print("   ```")
                    print("   Then re-run this cell.")
                    print("\n2. OR SET USE_PUBLIC_DATA = False:")
                    print("   Use only manual data for training")
                    raise
                else:
                    raise

print("\n" + "=" * 60)

EXTRACTING HAGRID SUBSET (OPTIMIZED)
Samples per gesture: 1500
Stratified sampling: True

✓ HaGRID subset already extracted to /content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/python_training/hagrid_subset
Skipping extraction...



In [13]:
# ============================================================
# CELL 6: Process Manual Images from Folder Structure
# ============================================================

import mediapipe as mp
import json
import os
from datetime import datetime
from tqdm import tqdm
from PIL import Image as PILImage
import numpy as np

# Manual data configuration
MANUAL_IMAGES_DIR = os.path.join(PROJECT_ROOT, "data/hand_gestures_manual")
MANUAL_JSON_DIR = os.path.join(PROJECT_ROOT, "data/hand_gestures_manual_json")

print("=" * 60)
print("🔄 PROCESSING MANUAL IMAGES FROM FOLDERS")
print("=" * 60)
print(f"\nInput: {MANUAL_IMAGES_DIR}")
print(f"Output: {MANUAL_JSON_DIR}")
print(f"\nExpected folder structure:")
print(f"  {MANUAL_IMAGES_DIR}/")
print(f"    ├── call/       (image files)")
print(f"    ├── like/       (image files)")
print(f"    ├── dislike/    (image files)")
print(f"    ├── fist/       (image files)")
print(f"    ├── palm/       (image files)")
print(f"    ├── peace/      (image files)")
print(f"    └── rock/       (image files)")


def process_manual_images():
    """
    Process all images from gesture folders and extract MediaPipe landmarks.

    Folder structure:
        data/hand_gestures_manual/
            call/
                img1.jpg
                img2.jpg
                ...
            like/
                img1.jpg
                ...

    Output:
        data/hand_gestures_manual_json/
            call_manual_001.json
            call_manual_002.json
            ...
    """

    # Create output directory
    os.makedirs(MANUAL_JSON_DIR, exist_ok=True)

    # Initialize MediaPipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=1,
        min_detection_confidence=0.5,
    )

    # Supported image extensions
    IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

    # Check if manual images directory exists
    if not os.path.exists(MANUAL_IMAGES_DIR):
        print(f"\n⚠️ Manual images directory not found: {MANUAL_IMAGES_DIR}")
        print("\nPlease create the folder structure and add your images:")
        print(f"  1. Create folder: {MANUAL_IMAGES_DIR}")
        print("  2. Create subfolders for each gesture (call, like, dislike, etc.)")
        print("  3. Add image files to each gesture folder")
        print("  4. Run this cell again")
        return

    # Find all gesture folders
    gesture_folders = [
        d
        for d in os.listdir(MANUAL_IMAGES_DIR)
        if os.path.isdir(os.path.join(MANUAL_IMAGES_DIR, d))
    ]

    if not gesture_folders:
        print(f"\n⚠️ No gesture folders found in {MANUAL_IMAGES_DIR}")
        print("\nCreate subfolders like: call/, like/, dislike/, etc.")
        return

    print(f"\n📁 Found {len(gesture_folders)} gesture folders: {gesture_folders}")

    total_stats = {
        "total_images": 0,
        "successful": 0,
        "no_hand": 0,
        "error": 0,
    }

    gesture_stats = {}

    # Process each gesture folder
    for gesture_name in gesture_folders:
        gesture_path = os.path.join(MANUAL_IMAGES_DIR, gesture_name)

        # Find all image files in this folder
        image_files = [
            f
            for f in os.listdir(gesture_path)
            if os.path.splitext(f.lower())[1] in IMAGE_EXTENSIONS
        ]

        if not image_files:
            print(f"\n⚠️ No images found in {gesture_name}/ folder")
            continue

        print(f"\n{'=' * 60}")
        print(f"📸 Processing: {gesture_name.upper()}")
        print(f"{'=' * 60}")
        print(f"Found {len(image_files)} images")

        successful = 0
        no_hand = 0
        error = 0

        # Process each image
        for img_file in tqdm(image_files, desc=f"  Processing {gesture_name}"):
            img_path = os.path.join(gesture_path, img_file)

            try:
                # Load image
                image = PILImage.open(img_path)
                img_np = np.array(image.convert("RGB"))

                # Process with MediaPipe
                results = hands.process(img_np)

                if not results.multi_hand_landmarks:
                    no_hand += 1
                    continue

                # Extract landmarks
                lm = results.multi_hand_landmarks[0]
                flat = []
                for p in lm.landmark:
                    flat.extend([p.x, p.y, p.z])

                # Generate unique filename
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                json_filename = f"{gesture_name}_manual_{timestamp}.json"
                json_path = os.path.join(MANUAL_JSON_DIR, json_filename)

                # Save to JSON
                json_data = {
                    "gesture": gesture_name,
                    "landmarks": flat,
                    "timestamp": timestamp,
                    "source": "manual_folder",
                    "original_file": img_file,
                    "original_path": img_path,
                }

                with open(json_path, "w") as f:
                    json.dump(json_data, f, indent=2)

                successful += 1

            except Exception:
                error += 1
                # Uncomment to see errors:
                # print(f"  ❌ Error processing {img_file}: {e}")
                continue

        # Update stats
        total_stats["total_images"] += len(image_files)
        total_stats["successful"] += successful
        total_stats["no_hand"] += no_hand
        total_stats["error"] += error

        gesture_stats[gesture_name] = {
            "total": len(image_files),
            "successful": successful,
            "no_hand": no_hand,
            "error": error,
            "success_rate": (successful / len(image_files) * 100)
            if len(image_files) > 0
            else 0,
        }

        # Print summary for this gesture
        print(f"  ✅ Successful: {successful}")
        print(f"  ❌ No hand detected: {no_hand}")
        print(f"  ⚠️  Errors: {error}")
        print(f"  📊 Success rate: {gesture_stats[gesture_name]['success_rate']:.1f}%")

    hands.close()

    # Overall summary
    print(f"\n\n{'=' * 60}")
    print("✅ PROCESSING COMPLETE")
    print(f"{'=' * 60}")

    print("\n📊 Overall Statistics:")
    print(f"  Total images processed: {total_stats['total_images']}")
    print(f"  ✅ Successful: {total_stats['successful']}")
    print(f"  ❌ No hand detected: {total_stats['no_hand']}")
    print(f"  ⚠️  Errors: {total_stats['error']}")

    if total_stats["total_images"] > 0:
        success_rate = total_stats["successful"] / total_stats["total_images"] * 100
        print(f"  📈 Overall success rate: {success_rate:.1f}%")

    print("\n📦 Per-Gesture Statistics:")
    for gesture_name, stats in gesture_stats.items():
        status = "✅" if stats["success_rate"] > 80 else "⚠️" if stats["success_rate"] > 50 else "❌"
        print(
            f"  {status} {gesture_name:12s}: "
            f"{stats['successful']:4d}/{stats['total']:4d} "
            f"({stats['success_rate']:.1f}%)"
        )

    print(f"\n💾 JSON files saved to: {MANUAL_JSON_DIR}")
    print(f"   Total JSON files created: {total_stats['successful']}")

    # Tips for improving success rate
    if total_stats["no_hand"] > 0:
        print("\n💡 Tips to improve success rate:")
        print(f"   • {total_stats['no_hand']} images had no hand detected")
        print("   • Make sure hand is clearly visible and well-lit")
        print("   • Avoid blurry or dark images")
        print("   • Show the full hand gesture in frame")

    return gesture_stats


# Run processing
print("\n🚀 Starting automatic processing...")
process_manual_images()

🔄 PROCESSING MANUAL IMAGES FROM FOLDERS

Input: /content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/python_training/data/hand_gestures_manual
Output: /content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/python_training/data/hand_gestures_manual_json

Expected folder structure:
  /content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/python_training/data/hand_gestures_manual/
    ├── call/       (image files)
    ├── like/       (image files)
    ├── dislike/    (image files)
    ├── fist/       (image files)
    ├── palm/       (image files)
    ├── peace/      (image files)
    └── rock/       (image files)

🚀 Starting automatic processing...

📁 Found 7 gesture folders: ['call', 'dislike', 'fist', 'palm', 'peace', 'rock', 'like']

📸 Processing: CALL
Found 284 images


  Processing call:   0%|          | 0/284 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
  Processing call: 100%|██████████| 284/284 [01:29<00:00,  3.17it/s]


  ✅ Successful: 281
  ❌ No hand detected: 3
  ⚠️  Errors: 0
  📊 Success rate: 98.9%

📸 Processing: DISLIKE
Found 183 images


  Processing dislike: 100%|██████████| 183/183 [00:33<00:00,  5.48it/s]


  ✅ Successful: 183
  ❌ No hand detected: 0
  ⚠️  Errors: 0
  📊 Success rate: 100.0%

📸 Processing: FIST
Found 191 images


...

  Processing rock: 100%|██████████| 440/440 [01:24<00:00,  5.18it/s]


  ✅ Successful: 428
  ❌ No hand detected: 12
  ⚠️  Errors: 0
  📊 Success rate: 97.3%

📸 Processing: LIKE
Found 255 images


  Processing like: 100%|██████████| 255/255 [00:47<00:00,  5.39it/s]

  ✅ Successful: 252
  ❌ No hand detected: 3
  ⚠️  Errors: 0
  📊 Success rate: 98.8%


✅ PROCESSING COMPLETE

📊 Overall Statistics:
  Total images processed: 1736
  ✅ Successful: 1704
  ❌ No hand detected: 32
  ⚠️  Errors: 0
  📈 Overall success rate: 98.2%

📦 Per-Gesture Statistics:
  ✅ call        :  281/ 284 (98.9%)
  ✅ dislike     :  183/ 183 (100.0%)
  ✅ fist        :  188/ 191 (98.4%)
  ✅ palm        :  150/ 150 (100.0%)
  ✅ peace       :  222/ 233 (95.3%)
  ✅ rock        :  428/ 440 (97.3%)
  ✅ like        :  252/ 255 (98.8%)

💾 JSON files saved to: /content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/python_training/data/hand_gestures_manual_json
   Total JSON files created: 1704

💡 Tips to improve success rate:
   • 32 images had no hand detected
   • Make sure hand is clearly visible and well-lit
   • Avoid blurry or dark images
   • Show the full hand gesture in frame


{'call': {'total': 284,
  'successful': 281,
  'no_hand': 3,
  'error': 0,
  'success_rate': 98.94366197183099},
 'dislike': {'total': 183,
  'successful': 183,
  'no_hand': 0,
  'error': 0,
  'success_rate': 100.0},
 'fist': {'total': 191,
  'successful': 188,
  'no_hand': 3,
  'error': 0,
  'success_rate': 98.42931937172776},
 'palm': {'total': 150,
  'successful': 150,
  'no_hand': 0,
  'error': 0,
  'success_rate': 100.0},
 'peace': {'total': 233,
  'successful': 222,
  'no_hand': 11,
  'error': 0,
  'success_rate': 95.27896995708154},
 'rock': {'total': 440,
  'successful': 428,
  'no_hand': 12,
  'error': 0,
  'success_rate': 97.27272727272728},
 'like': {'total': 255,
  'successful': 252,
  'no_hand': 3,
  'error': 0,
  'success_rate': 98.82352941176471}}

In [14]:
# ============================================================
# CELL 7: Extract MediaPipe Landmarks with Quality Filtering
# ============================================================

import cv2
import mediapipe as mp
from datetime import datetime
import json
import glob
import os
from tqdm import tqdm

# Skip if not using public data
if not USE_PUBLIC_DATA:
    print("=" * 60)
    print("⏭️  SKIPPING HAGRID LANDMARK EXTRACTION")
    print("=" * 60)
    print("USE_PUBLIC_DATA = False, skipping HaGRID processing")
    print("\n✓ Moving to next step...")
else:
    print("=" * 60)
    print("🔧 PROCESSING HAGRID WITH MEDIAPIPE + QUALITY FILTERING")
    print("=" * 60)

    HAGRID_IMAGES_ROOT = HAGRID_SUBSET_DIR

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    HAND_JSON_DIR = os.path.join(PROJECT_ROOT, "data/hand_gestures_public")
    os.makedirs(HAND_JSON_DIR, exist_ok=True)

    stats = {
        g: {
            "processed": 0,
            "failed_no_hand": 0,
            "filtered_too_small": 0,
            "filtered_too_large": 0,
            "filtered_off_center": 0,
            "filtered_low_confidence": 0
        }
        for g in HAND_GESTURES.keys()
    }

    for gesture in HAND_GESTURES.keys():
        gesture_dir = os.path.join(HAGRID_IMAGES_ROOT, gesture)

        if not os.path.isdir(gesture_dir):
            print(f"⚠️ Skipping '{gesture}': folder not found at {gesture_dir}")
            continue

        image_paths = sorted(glob.glob(os.path.join(gesture_dir, "*.jpeg")))

        if not image_paths:
            print(f"⚠️ No images found in {gesture_dir}")
            continue

        print(f"\n{gesture}: processing {len(image_paths)} images with quality filtering")

        for img_path in tqdm(image_paths, desc=f"MediaPipe {gesture}"):
            img = cv2.imread(img_path)
            if img is None:
                stats[gesture]["failed_no_hand"] += 1
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)

            if not results.multi_hand_landmarks:
                stats[gesture]["failed_no_hand"] += 1
                continue

            lm = results.multi_hand_landmarks[0]

            # NEW: Quality filtering
            # Check 1: Hand size (not too small or large)
            x_coords = [p.x for p in lm.landmark]
            y_coords = [p.y for p in lm.landmark]
            hand_width = max(x_coords) - min(x_coords)
            hand_height = max(y_coords) - min(y_coords)

            if hand_width < 0.15 or hand_height < 0.15:
                stats[gesture]["filtered_too_small"] += 1
                continue

            if hand_width > 0.85 or hand_height > 0.85:
                stats[gesture]["filtered_too_large"] += 1
                continue

            # Check 2: Hand position (reasonably centered)
            hand_center_x = (max(x_coords) + min(x_coords)) / 2
            hand_center_y = (max(y_coords) + min(y_coords)) / 2

            if hand_center_x < 0.15 or hand_center_x > 0.85:
                stats[gesture]["filtered_off_center"] += 1
                continue

            if hand_center_y < 0.15 or hand_center_y > 0.85:
                stats[gesture]["filtered_off_center"] += 1
                continue

            # Check 3: Landmark confidence (MediaPipe provides this)
            # Average z-coordinate shouldn't be too far (indicates poor tracking)
            z_coords = [p.z for p in lm.landmark]
            avg_z = sum(z_coords) / len(z_coords)

            if abs(avg_z) > 0.3:  # Too far forward/backward
                stats[gesture]["filtered_low_confidence"] += 1
                continue

            # Passed all quality checks - extract landmarks
            flat = []
            for p in lm.landmark:
                flat.extend([p.x, p.y, p.z])

            data = {
                "gesture": gesture,
                "landmarks": flat,
                "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
                "source": "hagrid",
                "image_file": os.path.basename(img_path),
                "quality": {
                    "hand_width": float(hand_width),
                    "hand_height": float(hand_height),
                    "center_x": float(hand_center_x),
                    "center_y": float(hand_center_y),
                    "avg_z": float(avg_z)
                }
            }

            out_name = f"{gesture}_{os.path.splitext(os.path.basename(img_path))[0]}.json"
            out_path = os.path.join(HAND_JSON_DIR, out_name)

            with open(out_path, "w") as f:
                json.dump(data, f, indent=2)

            stats[gesture]["processed"] += 1

    hands.close()

    print("\n" + "=" * 60)
    print("✅ HAND GESTURE PROCESSING SUMMARY (WITH QUALITY FILTERING)")
    print("=" * 60)

    for g, c in stats.items():
        total_attempted = (c["processed"] + c["failed_no_hand"] + c["filtered_too_small"] +
                          c["filtered_too_large"] + c["filtered_off_center"] + c["filtered_low_confidence"])
        if total_attempted == 0:
            continue

        success_rate = 100 * c["processed"] / total_attempted if total_attempted > 0 else 0

        print(f"\n{g:15s}:")
        print(f"  ✅ Accepted: {c['processed']:4d} ({success_rate:.1f}%)")
        print(f"  ❌ No hand:  {c['failed_no_hand']:4d}")
        print(f"  🔍 Filters:")
        print(f"     Too small:    {c['filtered_too_small']:4d}")
        print(f"     Too large:    {c['filtered_too_large']:4d}")
        print(f"     Off-center:   {c['filtered_off_center']:4d}")
        print(f"     Low conf:     {c['filtered_low_confidence']:4d}")

    print("\n" + "=" * 60)
    print(f"JSON output dir: {HAND_JSON_DIR}")

print("\n" + "=" * 60)

🔧 PROCESSING HAGRID WITH MEDIAPIPE + QUALITY FILTERING

call: processing 1500 images with quality filtering


MediaPipe call: 100%|██████████| 1500/1500 [06:22<00:00,  3.92it/s]



like: processing 1500 images with quality filtering


MediaPipe like: 100%|██████████| 1500/1500 [05:57<00:00,  4.19it/s]



dislike: processing 1500 images with quality filtering


...


peace: processing 1500 images with quality filtering


MediaPipe peace: 100%|██████████| 1500/1500 [06:08<00:00,  4.07it/s]



rock: processing 1584 images with quality filtering


MediaPipe rock: 100%|██████████| 1584/1584 [06:52<00:00,  3.84it/s]


✅ HAND GESTURE PROCESSING SUMMARY (WITH QUALITY FILTERING)

call           :
  ✅ Accepted:  861 (57.4%)
  ❌ No hand:   594
  🔍 Filters:
     Too small:      37
     Too large:       6
     Off-center:      0
     Low conf:        2

like           :
  ✅ Accepted:  837 (55.8%)
  ❌ No hand:   562
  🔍 Filters:
     Too small:      94
     Too large:       4
     Off-center:      0
     Low conf:        3

dislike        :
  ✅ Accepted:  803 (53.5%)
  ❌ No hand:   537
  🔍 Filters:
     Too small:     154
     Too large:       1
     Off-center:      0
     Low conf:        5

fist           :
  ✅ Accepted:  728 (48.5%)
  ❌ No hand:   516
  🔍 Filters:
     Too small:     254
     Too large:       0
     Off-center:      0
     Low conf:        2

palm           :
  ✅ Accepted: 1086 (72.4%)
  ❌ No hand:   358
  🔍 Filters:
     Too small:      28
     Too large:      27
     Off-center:      0
     Low conf:        1

peace          :
  ✅ Accepted:  842 (56.1%)
  ❌ No hand:   416
  🔍 Filters

In [16]:
# ============================================================
# CELL 8: Data Preprocessing (with Manual + Public Support)
# ============================================================

import os
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.model_selection import StratifiedShuffleSplit


def simple_train_val_test_split(X, y, train_ratio=0.7, val_ratio=0.15, seed=42):
    """Stratified split ensures each class is proportionally represented."""
    assert len(X) == len(y)

    # First split: train vs (val+test)
    sss1 = StratifiedShuffleSplit(n_splits=1, train_size=train_ratio, random_state=seed)
    train_idx, temp_idx = next(sss1.split(X, y))

    X_train, y_train = X[train_idx], y[train_idx]
    X_temp, y_temp = X[temp_idx], y[temp_idx]

    # Second split: val vs test
    val_ratio_adjusted = val_ratio / (val_ratio + (1 - train_ratio - val_ratio))
    sss2 = StratifiedShuffleSplit(
        n_splits=1, train_size=val_ratio_adjusted, random_state=seed
    )
    val_idx, test_idx = next(sss2.split(X_temp, y_temp))

    X_val, y_val = X_temp[val_idx], y_temp[val_idx]
    X_test, y_test = X_temp[test_idx], y_temp[test_idx]

    return X_train, X_val, X_test, y_train, y_val, y_test


class DatasetBuilder:
    def __init__(self, json_dirs, gesture_type):
        """
        Args:
            json_dirs: list of directories containing JSON files
            gesture_type: "hand"
        """
        self.json_dirs = json_dirs if isinstance(json_dirs, list) else [json_dirs]
        self.gesture_type = gesture_type
        self.class_names = []
        self.feature_mean = None
        self.feature_std = None

    def _load_single_file(self, path):
        with open(path, "r") as f:
            sample = json.load(f)

        gesture = sample.get("gesture")
        landmarks = sample.get("landmarks")

        # Filter out unwanted gestures for hand dataset
        if self.gesture_type == "hand" and gesture not in ALLOWED_HAND_GESTURES:
            return None, None

        # landmarks must be a flat list of numbers
        if isinstance(landmarks, dict):
            return None, None

        if not isinstance(landmarks, (list, tuple)):
            return None, None

        # Flatten nested lists if any
        flat = []
        for v in landmarks:
            if isinstance(v, (list, tuple)):
                flat.extend(v)
            else:
                flat.append(v)

        # Ensure all are numeric
        try:
            flat = [float(x) for x in flat]
        except Exception:
            return None, None

        return gesture, flat

    def load_jsons(self):
        """Load JSONs from all configured directories."""
        data = []
        labels = []

        for json_dir in self.json_dirs:
            if not os.path.isdir(json_dir):
                print(f"⚠️ JSON dir not found: {json_dir}, skipping")
                continue

            files = [f for f in os.listdir(json_dir) if f.endswith(".json")]
            print(f"Found {len(files)} JSON files in {json_dir}")

            skipped = 0
            before = len(data)

            for fname in tqdm(files, desc=f"Loading from {os.path.basename(json_dir)}"):
                path = os.path.join(json_dir, fname)
                gesture, flat = self._load_single_file(path)
                if gesture is None or flat is None:
                    skipped += 1
                    continue

                data.append(flat)
                labels.append(gesture)

            loaded_here = len(data) - before
            print(
                f"Loaded {loaded_here} samples from this directory, "
                f"skipped {skipped} bad files"
            )

        if len(data) == 0:
            print(f"⚠️ No valid data loaded for {self.gesture_type}")
            return np.empty((0,)), np.empty((0,))

        X = np.array(data, dtype=np.float32)
        y = np.array(labels, dtype=object)
        return X, y

    def augment_landmarks(self, X, y, augment_factor=0):
        """Create augmented copies with varied perturbations."""
        aug_X = [X]  # Original data
        aug_y = [y]

        for _ in range(augment_factor):
            # Stronger random translation
            shift_x = np.random.uniform(-0.05, 0.05, size=(len(X), 21))
            shift_y = np.random.uniform(-0.05, 0.05, size=(len(X), 21))
            shift_z = np.random.uniform(-0.02, 0.02, size=(len(X), 21))

            X_shifted = X.copy().reshape(len(X), 21, 3)
            X_shifted[:, :, 0] += shift_x  # x coords
            X_shifted[:, :, 1] += shift_y  # y coords
            X_shifted[:, :, 2] += shift_z  # z coords
            X_shifted = X_shifted.reshape(len(X), 63)

            # Stronger Gaussian noise
            X_noisy = X_shifted + np.random.normal(0, 0.01, X_shifted.shape)

            # Random scaling
            scale = np.random.uniform(0.9, 1.1, size=(len(X), 1))
            X_scaled = X_noisy * scale

            # Random rotation around z-axis
            angle = np.random.uniform(-0.2, 0.2, size=(len(X),))
            cos_a = np.cos(angle)[:, None]
            sin_a = np.sin(angle)[:, None]
            X_rot = X_scaled.copy().reshape(len(X), 21, 3)
            x_new = X_rot[:, :, 0] * cos_a - X_rot[:, :, 1] * sin_a
            y_new = X_rot[:, :, 0] * sin_a + X_rot[:, :, 1] * cos_a
            X_rot[:, :, 0] = x_new
            X_rot[:, :, 1] = y_new
            X_rot = X_rot.reshape(len(X), 63)

            aug_X.append(X_rot)
            aug_y.append(y.copy())

        return np.vstack(aug_X), np.hstack(aug_y)

    def prepare_dataset(self):
        X, y = self.load_jsons()
        if X.size == 0:
            raise RuntimeError(f"No data loaded for gesture_type={self.gesture_type}")

        print(f"\n📊 Total raw samples loaded: {len(X)}")

        # Augment before normalization
        X_aug, y_aug = self.augment_landmarks(X, y, augment_factor=0)
        print(f"📊 After augmentation: {len(X_aug)} samples")

        # Feature normalization
        self.feature_mean = X_aug.mean(axis=0, keepdims=True)
        self.feature_std = X_aug.std(axis=0, keepdims=True) + 1e-6
        X_norm = (X_aug - self.feature_mean) / self.feature_std

        # Class encoding
        self.class_names = sorted(list(set(y_aug)))
        print(f"\nClasses ({self.gesture_type}): {self.class_names}")

        label_to_idx = {label: idx for idx, label in enumerate(self.class_names)}
        y_encoded = np.array([label_to_idx[label] for label in y_aug], dtype=np.int64)

        X_train, X_val, X_test, y_train, y_val, y_test = simple_train_val_test_split(
            X_norm, y_encoded, train_ratio=0.7, val_ratio=0.15, seed=42
        )

        dataset = {
            "train_data": X_train,
            "train_labels": y_train,
            "val_data": X_val,
            "val_labels": y_val,
            "test_data": X_test,
            "test_labels": y_test,
            "class_names": self.class_names,
            "feature_mean": self.feature_mean.astype(np.float32),
            "feature_std": self.feature_std.astype(np.float32),
        }

        print(
            f"\n{self.gesture_type.upper()} dataset sizes → "
            f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}"
        )

        return dataset


# ============================================================
# Build dataset based on configuration flags
# ============================================================

print("=" * 60)
print("BUILDING HAND GESTURE DATASET")
print("=" * 60)

# Determine which directories to include
json_dirs_to_use = []

if USE_PUBLIC_DATA:
    public_dir = os.path.join(PROJECT_ROOT, "data/hand_gestures_public")
    if os.path.exists(public_dir):
        json_dirs_to_use.append(public_dir)
        print(f"✓ Including public HaGRID data from: {public_dir}")
    else:
        print(f"⚠️ Public data directory not found: {public_dir}")

if USE_MANUAL_DATA:
    manual_dir = os.path.join(PROJECT_ROOT, "data/hand_gestures_manual_json")
    if os.path.exists(manual_dir):
        json_dirs_to_use.append(manual_dir)
        print(f"✓ Including manual collected data from: {manual_dir}")
    else:
        print(f"⚠️ Manual data directory not found: {manual_dir}")

if len(json_dirs_to_use) == 0:
    raise RuntimeError(
        "❌ No data sources enabled! Set USE_PUBLIC_DATA or USE_MANUAL_DATA to True"
    )

print(f"\n📂 Total data sources: {len(json_dirs_to_use)}")

# Build combined dataset
hand_builder = DatasetBuilder(json_dirs_to_use, "hand")
hand_dataset = hand_builder.prepare_dataset()

# Save dataset
os.makedirs(os.path.join(PROJECT_ROOT, "data"), exist_ok=True)
with open(os.path.join(PROJECT_ROOT, "data/hand_gesture_dataset.pkl"), "wb") as f:
    pickle.dump(hand_dataset, f)

print("\n✓ Hand dataset saved")

BUILDING HAND GESTURE DATASET
✓ Including public HaGRID data from: /content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/python_training/data/hand_gestures_public
✓ Including manual collected data from: /content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/python_training/data/hand_gestures_manual_json

📂 Total data sources: 2
Found 9109 JSON files in /content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/python_training/data/hand_gestures_public


Loading from hand_gestures_public: 100%|██████████| 9109/9109 [01:54<00:00, 79.55it/s] 


Loaded 8691 samples from this directory, skipped 418 bad files
Found 15927 JSON files in /content/gdrive/MyDrive/Colab_Notebooks/HCI/Project/GestureTouchlessControl/python_training/data/hand_gestures_manual_json


Loading from hand_gestures_manual_json: 100%|██████████| 15927/15927 [06:20<00:00, 41.90it/s] 


Loaded 15927 samples from this directory, skipped 0 bad files

📊 Total raw samples loaded: 24618
📊 After augmentation: 24618 samples

Classes (hand): ['call', 'dislike', 'fist', 'like', 'palm', 'peace', 'rock']

HAND dataset sizes → Train: 17232, Val: 3692, Test: 3694

✓ Hand dataset saved


In [17]:
# =============================================================================
# 🔧 CELL 9: MODEL TRAINING
# =============================================================================

from tensorflow import keras
from tensorflow.keras import layers, regularizers


class GestureModelTrainer:
    """Unified trainer for gesture recognition models."""

    def __init__(self, dataset, model_name, strategy):
        """
        Args:
            dataset: dict with train/val/test data and labels
            model_name: str, name for saving
            strategy: tf.distribute.Strategy (from CELL 1B)
        """
        self.dataset = dataset
        self.model_name = model_name
        self.strategy = strategy
        self.model = None
        self.history = None

        # Extract dataset info
        self.class_names = dataset["class_names"]
        self.num_classes = len(self.class_names)

        # Determine input size from data shape
        self.input_size = dataset["train_data"].shape[1]

        print(f"Initialized trainer for '{model_name}'")
        print(f"  Input size: {self.input_size}")
        print(f"  Classes: {self.num_classes} → {self.class_names}")
        print(f"  Train samples: {len(dataset['train_data'])}")
        print(f"  Val samples: {len(dataset['val_data'])}")
        print(f"  Test samples: {len(dataset['test_data'])}")

    def build_model(self):
        """Build improved neural network with better architecture."""
        with self.strategy.scope():
            model = keras.Sequential(
                [
                    # Input layer
                    layers.Input(shape=(self.input_size,)),

                    # Layer 1: Feature extraction
                    layers.Dense(
                        256,
                        activation="relu",
                        kernel_regularizer=regularizers.l2(0.001),
                    ),
                    layers.BatchNormalization(),
                    layers.Dropout(0.3),

                    # Layer 2: Deep feature learning
                    layers.Dense(
                        128,
                        activation="relu",
                        kernel_regularizer=regularizers.l2(0.001),
                    ),
                    layers.BatchNormalization(),
                    layers.Dropout(0.3),

                    # Layer 3: Refined features
                    layers.Dense(
                        64,
                        activation="relu",
                        kernel_regularizer=regularizers.l2(0.001),
                    ),
                    layers.BatchNormalization(),
                    layers.Dropout(0.2),

                    # Output layer
                    layers.Dense(self.num_classes, activation="softmax"),
                ]
            )

            # Compile with improved settings
            model.compile(
                optimizer=keras.optimizers.Adam(learning_rate=0.001),
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"],
            )

        self.model = model
        print("\nModel architecture built:")
        model.summary()
        return model

    def train(self, epochs=100, batch_size=256):
        """Train the model with callbacks."""
        if self.model is None:
            self.build_model()

        # Prepare data
        X_train = self.dataset["train_data"]
        y_train = self.dataset["train_labels"]
        X_val = self.dataset["val_data"]
        y_val = self.dataset["val_labels"]

        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor="val_accuracy",
                patience=15,
                restore_best_weights=True,
                verbose=1,
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor="val_loss",
                factor=0.5,
                patience=5,
                min_lr=1e-6,
                verbose=1,
            ),
        ]

        print(f"\n{'=' * 60}")
        print(f"TRAINING {self.model_name.upper()}")
        print(f"{'=' * 60}")
        print(f"Epochs: {epochs}")
        print(f"Batch size: {batch_size}")
        print(f"Training on {len(X_train)} samples")
        print(f"Validating on {len(X_val)} samples")
        print(f"{'=' * 60}\n")

        # Train
        self.history = self.model.fit(
            X_train,
            y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            verbose=1,
        )

        # Evaluate on test set
        X_test = self.dataset["test_data"]
        y_test = self.dataset["test_labels"]

        test_loss, test_acc = self.model.evaluate(X_test, y_test, verbose=0)

        print(f"\n{'=' * 60}")
        print("TRAINING COMPLETE")
        print(f"{'=' * 60}")
        print(f"Final validation accuracy: {self.history.history['val_accuracy'][-1]:.4f}")
        print(f"Test accuracy: {test_acc:.4f}")
        print(f"{'=' * 60}\n")

        return self.history


# =============================================================================
# TRAIN THE MODEL
# =============================================================================

print("=" * 60)
print("CREATING AND TRAINING HAND GESTURE MODEL")
print("=" * 60)

# Create trainer
hand_trainer = GestureModelTrainer(
    dataset=hand_dataset,
    model_name="hand_gesture",
    strategy=strategy,  # from CELL 1B
)

# Train the model
history = hand_trainer.train(
    epochs=100,
    batch_size=256,
)

print("\n✅ Model training complete!")
print("   hand_trainer object created and ready")
print("   hand_trainer.model is the trained Keras model")

CREATING AND TRAINING HAND GESTURE MODEL
Initialized trainer for 'hand_gesture'
  Input size: 63
  Classes: 7 → ['call', 'dislike', 'fist', 'like', 'palm', 'peace', 'rock']
  Train samples: 17232
  Val samples: 3692
  Test samples: 3694

Model architecture built:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,783 (233.53 KB)

 Trainable params: 58,887 (230.03 KB)

 Non-trainable params: 896 (3.50 KB)


TRAINING HAND_GESTURE
Epochs: 100
Batch size: 256
Training on 17232 samples
Validating on 3692 samples

Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.5730 - loss: 1.6321 - val_accuracy: 0.9596 - val_loss: 1.0271 - learning_rate: 0.0010
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9542 - loss: 0.5317 - val_accuracy: 0.9756 - val_loss: 0.6035 - learning_rate: 0.0010
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9732 - loss: 0.4365 - val_accuracy: 0.9837 - val_loss: 0.4391 - learning_rate: 0.0010
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9768 - loss: 0.3917 - val_accuracy: 0.9875 - val_loss: 0.3575 - learning_rate: 0.0010
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9825 - loss: 0.3524 - val_accuracy: 0.9889 - val_loss: 0.3205 - learning_rate: 0.0010
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9826 - loss: 0.3260 - val_accuracy: 0.9878 - val_loss: 0.2936 - learn

In [18]:
# =============================================================================
# 🔧 CELL 10: SAVE MODEL (.tflite + .json with mean/std)
# =============================================================================

import json
import os
import shutil
import tensorflow as tf

print("=" * 60)
print("SAVING MODEL (.tflite) AND LABELS (.json)")
print("=" * 60)

# Create models directory
MODELS_DIR = os.path.join(PROJECT_ROOT, "models")
os.makedirs(MODELS_DIR, exist_ok=True)

# ============================================================================
# STEP 1: Convert Keras model to TFLite
# ============================================================================

print("\n📦 Converting Keras model to TFLite...")

converter = tf.lite.TFLiteConverter.from_keras_model(hand_trainer.model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

tflite_path = os.path.join(MODELS_DIR, "hand_gesture_model.tflite")
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

tflite_size_kb = len(tflite_model) / 1024.0

print(f"✅ TFLite model saved to: {tflite_path}")
print(f"   Model size: {tflite_size_kb:.2f} KB")

# ============================================================================
# STEP 2: Save labels with normalization parameters (CRITICAL FOR ANDROID)
# ============================================================================

print("\n📋 Saving gesture labels with normalization parameters...")

# CRITICAL: HandGestureClassifier.kt expects feature_mean and feature_std!
labels = hand_dataset["class_names"]
labels_data = {
    "labels": labels,
    "feature_mean": hand_dataset["feature_mean"].flatten().tolist(),
    "feature_std": hand_dataset["feature_std"].flatten().tolist(),
}

labels_path = os.path.join(MODELS_DIR, "gesture_labels.json")
with open(labels_path, "w") as f:
    json.dump(labels_data, f, indent=2)

print(f"✅ Labels saved to: {labels_path}")
print(f"   Classes: {labels_data['labels']}")
print(f"   Feature mean len: {len(labels_data['feature_mean'])} (should be 63)")
print(f"   Feature std len:  {len(labels_data['feature_std'])} (should be 63)")

# ============================================================================
# STEP 3: Android-friendly copies
# ============================================================================

print("\n📱 Creating Android-compatible file names...")

android_model_path = os.path.join(MODELS_DIR, "hand_model.tflite")
android_labels_path = os.path.join(MODELS_DIR, "hand_labels.json")

shutil.copy(tflite_path, android_model_path)
shutil.copy(labels_path, android_labels_path)

print(f"✅ Android model:  {android_model_path}")
print(f"✅ Android labels: {android_labels_path}")

# ============================================================================
# STEP 4: Verification summary
# ============================================================================

# Evaluate test accuracy
test_loss, test_acc = hand_trainer.model.evaluate(
    hand_dataset["test_data"],
    hand_dataset["test_labels"],
    verbose=0,
)

total_samples = (
    len(hand_dataset["train_data"])
    + len(hand_dataset["val_data"])
    + len(hand_dataset["test_data"])
)

dataset_type = []
if USE_PUBLIC_DATA:
    dataset_type.append("Public")
if USE_MANUAL_DATA:
    dataset_type.append("Manual")
dataset_str = " + ".join(dataset_type) if dataset_type else "Unknown"

print(f"\n{'=' * 60}")
print("✅ MODEL EXPORT COMPLETE!")
print(f"{'=' * 60}\n")

print(f"📁 Files saved in: {MODELS_DIR}\n")
print("Training Files (descriptive names):")
print(f"  • hand_gesture_model.tflite  ({tflite_size_kb:.2f} KB)")
print("  • gesture_labels.json\n")
print("Android Files (expected names):")
print(f"  • hand_model.tflite          ({tflite_size_kb:.2f} KB)")
print("  • hand_labels.json\n")

print("📊 Model Details:")
print("  • Input shape: (1, 63)  [21 landmarks × 3 coordinates]")
print(
    f"  • Output shape: (1, {len(labels_data['labels'])})  "
    f"[{len(labels_data['labels'])} gesture classes]"
)
print(f"  • Gesture classes: {', '.join(labels_data['labels'])}")
print(f"  • Trained on: {dataset_str} data")
print(f"  • Total samples: {total_samples}")
print(f"  • Test accuracy: {test_acc:.4f}\n")

print("⚙️  JSON Structure (for Android):")
print("{")
print(f"  \"labels\": [...],        # List of {len(labels_data['labels'])} gesture names")
print("  \"feature_mean\": [...],  # 63 normalization means")
print("  \"feature_std\": [...]    # 63 normalization stds")
print("}\n")

print("📱 Next Steps:")
print(f"  1. Download both files from: {MODELS_DIR}/")
print("  2. Copy to your Android project: /src/main/assets/")
print("     • hand_model.tflite")
print("     • hand_labels.json")
print("  3. HandGestureClassifier.kt will automatically:")
print("     • Load the model")
print("     • Load normalization parameters")
print("     • Preprocess landmarks with correct mean/std")
print("     • Predict gestures accurately\n")

print("🔧 The preprocessing in Python MATCHES Android exactly:")
print("   normalized = (landmarks - feature_mean) / (feature_std + 1e-6)\n")

print("=" * 60)

SAVING MODEL (.tflite) AND LABELS (.json)

📦 Converting Keras model to TFLite...
Saved artifact at '/tmp/tmpl_b3gij6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 63), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  132291196125008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132291196124816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132291100444816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132291100445008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132291100443472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132291100443664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132291196123664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132291100445776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132291100446352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132291100446